<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt
! pip install -U git+https://github.com/mariostoev/finviz
! pip install "tqdm==4.43.0"

fatal: destination path 'StockSellRulesChecker' already exists and is not an empty directory.
  Cloning https://github.com/mariostoev/finviz to /tmp/pip-req-build-8iri40fz
  Running command git clone -q https://github.com/mariostoev/finviz /tmp/pip-req-build-8iri40fz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=16979 sha256=99e7134bcf928d4844e5d47af57e21baa27126fe1da1fe5a245988968f821166
  Stored in directory: /tmp/pip-ephem-wheel-cache-9pjc1s2g/wheels/98/cc/6a/68893eb662f0f196f8e153e9ce4424a8ae9e1f022fd64e4d6d
Successfully built finviz
  Found existing installation: finviz 1.3.4
    Uninstalling finviz-1.3.4:
      Successfully uninstalled finviz-1.3.4


In [1]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

import time
from tqdm.contrib.concurrent import process_map


In [18]:
eps_5year_over0pct = False
eps_qoq_over0pct = False
eps_yoy_over0pct = False
sales_5years_over0pct = False
sales_qoq_over0pct = True

filters = []
if eps_5year_over0pct:
  filters.append('fa_eps5years_pos')
if eps_qoq_over0pct:
  filters.append('fa_epsqoq_pos')
if eps_yoy_over0pct:
  filters.append('fa_epsyoy_pos')
if sales_5years_over0pct:
  filters.append('fa_sales5years_pos')
if sales_qoq_over0pct:
  filters.append('fa_salesqoq_pos')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [19]:
print(len(stock_list.data))

2456


In [20]:

def moving_average(yahoo_df, days, delta=0):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max() - datetime.timedelta(days=delta)
    end_date = curr_date - datetime.timedelta(days=days) - datetime.timedelta(days=delta)
    
    df_days = df[df['Date'] >= end_date]
    df_days = df_days[df_days['Date'] <= curr_date]
    return float(df_days['Close'].mean())

def moving_average_volume(yahoo_df, days, delta=0):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max() - datetime.timedelta(days=delta)
    end_date = curr_date - datetime.timedelta(days=days) - datetime.timedelta(days=delta)
    
    df_days = df[df['Date'] >= end_date]
    df_days = df_days[df_days['Date'] <= curr_date]
    return float(df_days['Volume'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())

def SMA200_slope_positive_rule(yahoo_df, ticker, days=21):
  for day in range(days):
    if moving_average(yahoo_df, days=200, delta=day) >= moving_average(yahoo_df, days=200, delta=day+1):
      continue
    else:
      return False
  return True

def screen_stock(stock):
  if stock["Ticker"] == "":
    return

  finviz_stats = finviz.get_stock(stock['Ticker'])
  prev_close = float(finviz_stats['Prev Close'].replace("$",""))

  # Price greater than $10 rule
  if prev_close < 10:
    return

  screened_stocks[stock['Ticker']] = {}
  
  try:
      yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
  except:
    return

  SMA200_value = moving_average(yahoo_df, days=200)
  SMA150_value = moving_average(yahoo_df, days=150)
  SMA50_value = moving_average(yahoo_df, days=50)
  SMA50_volume_value = moving_average_volume(yahoo_df, days=50)

  # Liquidity Rule
  if SMA50_value*SMA50_volume_value <= 20e6:
    return

  SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))/100
  SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))/100
  volume = float(finviz_stats['Volume'].replace(",",""))

  try:
    EPS_QoQ_percent = float(finviz_stats['EPS Q/Q'].replace("%",""))/100
  except:
    EPS_QoQ_percent = 0

  try:
    Sales_QoQ_percent = float(finviz_stats['Sales Q/Q'].replace("%",""))/100
  except:
    Sales_QoQ_percent = 0

  try:
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))/100
  except:
    inst_own = 0
  
  shares_outstanding = finviz_stats['Shs Outstand']
  if 'M' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
  elif 'B' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
  elif 'T' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12
  else:
    shares_outstanding = 0

  week52_high, week52_low = week52_low_high(yahoo_df)
  
  screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
  screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
  screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
  screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
  screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
  screened_stocks[stock['Ticker']]['EPS_QoQ_percent'] = EPS_QoQ_percent
  screened_stocks[stock['Ticker']]['Sales_QoQ_percent'] = Sales_QoQ_percent
  screened_stocks[stock['Ticker']]['prev_close'] = prev_close
  screened_stocks[stock['Ticker']]['week52_high'] = week52_high
  screened_stocks[stock['Ticker']]['week52_low'] = week52_low
  screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
  screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
  screened_stocks[stock['Ticker']]['volume'] = volume

  # volume*price > $15mm rule
  if volume*prev_close > 15000000:
    vol_price_rule = True
  else:
    vol_price_rule = False
  screened_stocks[stock['Ticker']]['vol_price_rule'] = vol_price_rule

  # EPS QoQ Yearly > 18% rule
  if EPS_QoQ_percent >= .18:
    eps_QoQ_yearly_rule = True
  else:
    eps_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['eps_QoQ_yearly_rule'] = eps_QoQ_yearly_rule

  # Sales QoQ Yearly > 25% rule
  if Sales_QoQ_percent >= .25:
    sales_QoQ_yearly_rule = True
  else:
    sales_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['sales_QoQ_yearly_rule'] = sales_QoQ_yearly_rule

  # Shares Outstanding <= 25 mil
  if shares_outstanding <= 25e6:
    shares_outstanding_rule = True
  else:
    shares_outstanding_rule = False
  screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

  # Institutional Ownership < 35%
  if 0.05 <= inst_own <= .35:
    inst_ownership_rule = True
  else:
    inst_ownership_rule = False
  screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

  # Positive 200d MA positive
  SMA200_slope_rule = SMA200_slope_positive_rule(yahoo_df, ticker=stock['Ticker'], days=21)
  screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
      
  # 150d MA greater than 200d MA
  if SMA150_value > SMA200_value:
      SMA150_greater_SMA200_rule = True
  else:
      SMA150_greater_SMA200_rule = False
  screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
      
  # 50d MA greater than 150d MA
  if SMA50_value > SMA150_value:
      SMA50_greater_SMA150_rule = True
  else:
      SMA50_greater_SMA150_rule = False
  screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
      
  # Close above 50d MA
  if prev_close > SMA50_value:
      close_greater_SMA50_rule = True
  else:
      close_greater_SMA50_rule = False
  screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
      
  # 52 week high low span rule
  if 0.75*week52_high > 1.25*week52_low:
      week52_span_rule = True
  else:
      week52_span_rule = False
  screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
  
  # Close above 52 week high - 25%
  if prev_close > 0.75*week52_high:
      close_above_52weekhigh_rule = True
  else:
      close_above_52weekhigh_rule = False
  screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule
  return screened_stocks

screened_stocks = {}
results = process_map(screen_stock, stock_list, max_workers=16)



    


In [21]:
screened_stocks = {}
for d in results:
  if d is not None:
    screened_stocks.update(d)
  
output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

In [22]:
primary_rules = ['eps_QoQ_yearly_rule', 'sales_QoQ_yearly_rule', 'SMA200_slope_rule', 
                 'SMA150_greater_SMA200_rule', 'SMA50_greater_SMA150_rule', 'close_greater_SMA50_rule', 
                 'week52_span_rule', 'close_above_52weekhigh_rule']

secondary_rules = ['vol_price_rule', 'inst_ownership_rule', 'shares_outstanding_rule']
df_out['Primary Passed Tests'] = (df_out[list(primary_rules)]).sum(1)
df_out['Secondary Passed Tests'] = (df_out[list(secondary_rules)]).sum(1)

cols = df_out.columns.tolist()

for rule in primary_rules:
  if rule in cols:
    idx = cols.index(rule)
    cols[idx] = rule + " (1st)"

for rule in secondary_rules:
  if rule in cols:
    idx = cols.index(rule)
    cols[idx] = rule + " (2nd)"

df_out.columns = cols

In [23]:
df_out.sort_values('Primary Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule (2nd),eps_QoQ_yearly_rule (1st),sales_QoQ_yearly_rule (1st),shares_outstanding_rule (2nd),inst_ownership_rule (2nd),SMA200_slope_rule (1st),SMA150_greater_SMA200_rule (1st),SMA50_greater_SMA150_rule (1st),close_greater_SMA50_rule (1st),week52_span_rule (1st),close_above_52weekhigh_rule (1st),Primary Passed Tests,Secondary Passed Tests
915,PLT,26.869197,31.337255,40.682500,0.7409,0.0701,1.242,0.261,43.09,44.720001,7.440000,0.985,4.125000e+07,475760.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
1618,ABT,112.060365,113.954117,121.151666,0.1231,0.0134,1.088,0.287,121.47,128.229996,62.820000,0.758,1.780000e+09,3782517.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
1119,UCTT,33.859343,37.885490,48.240556,0.6626,0.1223,3.087,0.291,55.09,56.919998,12.710000,0.906,4.050000e+07,574626.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
1118,APAM,46.644453,48.824216,50.360278,0.1966,0.0057,0.517,0.253,53.12,54.419998,18.059999,0.839,5.697000e+07,686377.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
1111,DT,43.326131,44.059902,51.617222,0.1908,0.0290,9.142,0.276,51.76,56.110001,20.650000,0.976,2.810100e+08,1852132.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,ESCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0
868,AMRK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0
424,CFBK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0
425,GSBD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0


In [24]:
df_out[df_out["Primary Passed Tests"] >= 8]

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule (2nd),eps_QoQ_yearly_rule (1st),sales_QoQ_yearly_rule (1st),shares_outstanding_rule (2nd),inst_ownership_rule (2nd),SMA200_slope_rule (1st),SMA150_greater_SMA200_rule (1st),SMA50_greater_SMA150_rule (1st),close_greater_SMA50_rule (1st),week52_span_rule (1st),close_above_52weekhigh_rule (1st),Primary Passed Tests,Secondary Passed Tests
1,AR,5.634161,6.472255,9.106111,0.8928,0.0843,1.162,0.374,10.02,10.560000,0.670000,0.7220,268670000.0,9323958.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
3,KOPN,3.908102,4.785000,9.388889,2.1376,0.2669,1.174,0.598,11.03,13.560000,0.250000,0.1840,81690000.0,4306055.0,True,True,True,False,True,True,True,True,True,True,True,8.0,2.0
88,HIMX,7.519270,8.859804,13.329167,1.0212,0.0632,31.824,0.577,14.35,15.950000,2.360000,0.1610,172180000.0,5207563.0,True,True,True,False,True,True,True,True,True,True,True,8.0,2.0
145,GTEC,6.614365,7.977647,11.158889,1.6695,0.4369,1.106,0.382,14.10,14.940000,1.500000,0.0110,10020000.0,123064.0,False,True,True,True,False,True,True,True,True,True,True,8.0,1.0
231,ABR,13.768467,14.495686,15.969445,0.2713,0.0286,1.049,0.416,16.41,16.969999,4.060000,0.4550,119920000.0,898631.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750,SITE,145.806205,153.473432,165.338889,0.2075,-0.0048,3.328,0.262,172.82,181.529999,57.189999,0.0000,44400000.0,316249.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
1753,MKSI,142.280804,153.010589,164.814446,0.2436,-0.0042,1.704,0.321,176.33,188.240005,73.410004,0.9804,55100000.0,521026.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
1765,QRVO,155.674306,164.401961,174.310276,0.2236,0.0040,0.276,0.260,183.28,186.029999,69.360001,0.8700,113810000.0,1022863.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
1814,GLOB,198.934964,203.949216,215.283890,0.1474,0.0078,0.287,0.262,215.99,232.110001,72.650002,0.9400,39720000.0,122749.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0


In [25]:
def check_double_bottom_chart_pattern(df):
  tickers = df['Ticker'].unique()
  try:
    stocks = Screener(tickers=tickers, filters=['ta_pattern_doublebottom'], table='Performance', order='price')
  except:
    stocks = []

  df['Double Bottom Chart'] = False
  for stock in stocks:
    df.loc[df.Ticker == stock['Ticker'], 'Double Bottom Chart'] = True
  return df

def check_inverse_head_and_shoulder_chart_pattern(df):
  tickers = df['Ticker'].unique()
  try:
    stocks = Screener(tickers=tickers, filters=['ta_pattern_headandshouldersinv'], table='Performance', order='price')
  except:
    stocks = []

  df['Inverse Head and Shoulder Pattern'] = False
  for stock in stocks:
    df.loc[df.Ticker == stock['Ticker'], 'Inverse Head and Shoulder Pattern'] = True
  return df

def check_multiple_bottom_chart_pattern(df):
  tickers = df['Ticker'].unique()
  try:
    stocks = Screener(tickers=tickers, filters=['ta_pattern_multiplebottom'], table='Performance', order='price')
  except:
    stocks = []

  df['Multiple Bottom Pattern'] = False
  for stock in stocks:
    df.loc[df.Ticker == stock['Ticker'], 'Multiple Bottom Pattern'] = True
  return df

def check_channel_up_chart_pattern(df):
  tickers = df['Ticker'].unique()
  try:
    stocks = Screener(tickers=tickers, filters=['ta_pattern_channelup'], table='Performance', order='price')
  except:
    stocks = []

  df['Channel Up Pattern'] = False
  for stock in stocks:
    df.loc[df.Ticker == stock['Ticker'], 'Channel Up Pattern'] = True
  return df

def check_channel_up_strong_chart_pattern(df):
  tickers = df['Ticker'].unique()
  try:
    stocks = Screener(tickers=tickers, filters=['ta_pattern_channelup2'], table='Performance', order='price')
  except:
    stocks = []

  df['Channel Up Strong Pattern'] = False
  for stock in stocks:
    df.loc[df.Ticker == stock['Ticker'], 'Channel Up Strong Pattern'] = True
  return df

def check_wedge_down_chart_pattern(df):
  tickers = df['Ticker'].unique()
  try:
    stocks = Screener(tickers=tickers, filters=['ta_pattern_wedgedown'], table='Performance', order='price')
  except:
    stocks = []

  df['Wedge Down Pattern'] = False
  for stock in stocks:
    df.loc[df.Ticker == stock['Ticker'], 'Wedge Down Pattern'] = True
  return df

def check_wedge_down_strong_chart_pattern(df):
  tickers = df['Ticker'].unique()
  try:
    stocks = Screener(tickers=tickers, filters=['ta_pattern_wedgedown2'], table='Performance', order='price')
  except:
    stocks = []

  df['Wedge Down Strong Pattern'] = False
  for stock in stocks:
    df.loc[df.Ticker == stock['Ticker'], 'Wedge Down Strong Pattern'] = True
  return df

df_passed = df_out[df_out["Primary Passed Tests"] >= 8]
df_passed = check_double_bottom_chart_pattern(df_passed)
df_passed = check_inverse_head_and_shoulder_chart_pattern(df_passed)
df_passed = check_multiple_bottom_chart_pattern(df_passed)
df_passed = check_channel_up_chart_pattern(df_passed)
df_passed = check_channel_up_strong_chart_pattern(df_passed)
df_passed = check_wedge_down_chart_pattern(df_passed)
df_passed = check_wedge_down_strong_chart_pattern(df_passed)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [27]:
df_passed

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule (2nd),eps_QoQ_yearly_rule (1st),sales_QoQ_yearly_rule (1st),shares_outstanding_rule (2nd),inst_ownership_rule (2nd),SMA200_slope_rule (1st),SMA150_greater_SMA200_rule (1st),SMA50_greater_SMA150_rule (1st),close_greater_SMA50_rule (1st),week52_span_rule (1st),close_above_52weekhigh_rule (1st),Primary Passed Tests,Secondary Passed Tests,Double Bottom Chart,Inverse Head and Shoulder Pattern,Multiple Bottom Pattern,Channel Up Pattern,Channel Up Strong Pattern,Wedge Down Pattern,Wedge Down Strong Pattern
1,AR,5.634161,6.472255,9.106111,0.8928,0.0843,1.162,0.374,10.02,10.560000,0.670000,0.7220,268670000.0,9323958.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0,False,False,False,False,False,False,False
3,KOPN,3.908102,4.785000,9.388889,2.1376,0.2669,1.174,0.598,11.03,13.560000,0.250000,0.1840,81690000.0,4306055.0,True,True,True,False,True,True,True,True,True,True,True,8.0,2.0,False,False,False,False,False,False,False
88,HIMX,7.519270,8.859804,13.329167,1.0212,0.0632,31.824,0.577,14.35,15.950000,2.360000,0.1610,172180000.0,5207563.0,True,True,True,False,True,True,True,True,True,True,True,8.0,2.0,False,False,False,False,False,False,False
145,GTEC,6.614365,7.977647,11.158889,1.6695,0.4369,1.106,0.382,14.10,14.940000,1.500000,0.0110,10020000.0,123064.0,False,True,True,True,False,True,True,True,True,True,True,8.0,1.0,False,False,False,False,False,False,False
231,ABR,13.768467,14.495686,15.969445,0.2713,0.0286,1.049,0.416,16.41,16.969999,4.060000,0.4550,119920000.0,898631.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750,SITE,145.806205,153.473432,165.338889,0.2075,-0.0048,3.328,0.262,172.82,181.529999,57.189999,0.0000,44400000.0,316249.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0,False,False,False,False,False,False,False
1753,MKSI,142.280804,153.010589,164.814446,0.2436,-0.0042,1.704,0.321,176.33,188.240005,73.410004,0.9804,55100000.0,521026.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0,False,False,False,True,False,False,False
1765,QRVO,155.674306,164.401961,174.310276,0.2236,0.0040,0.276,0.260,183.28,186.029999,69.360001,0.8700,113810000.0,1022863.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0,False,False,False,False,False,False,False
1814,GLOB,198.934964,203.949216,215.283890,0.1474,0.0078,0.287,0.262,215.99,232.110001,72.650002,0.9400,39720000.0,122749.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0,False,False,False,False,False,False,False
